# Notebook to update data for the forecast

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
os.getcwd()

'/home/fccoelho/Documentos/Projects_Software/transfer-learning-forecast/forecast'

## LSTM models for the states and "Macroregionais de saúde"

### Path where the cases and climate data are saved:  

In [2]:
PATH = '../data'

### The dataframe below will be used in the functions to get the link between the geocodes and the health macroregion code: 
    

In [3]:
dfs = pd.read_csv('../macro_saude.csv')

dfs.head()

,geocode,name_muni,name_region,code_region,name_macro,code_macro,state
0,1100015,Alta Floresta D'Oeste,Zona da Mata,11005,Cacoal,1101,RO
1,1100023,Ariquemes,Vale do Jamari,11001,Porto Velho,1102,RO
2,1100031,Cabixi,Cone Sul,11006,Cacoal,1101,RO
3,1100049,Cacoal,Café,11002,Cacoal,1101,RO
4,1100056,Cerejeiras,Cone Sul,11006,Cacoal,1101,RO


In [4]:
def get_geocodes_and_state(macro): 
    '''
    This function is used to get the geocodes and state that refer to a specific health macro region code
    
    :param macro:int. A four-digit number
        
    '''
    
    dfs = pd.read_csv('../macro_saude.csv')
    
    geocodes = dfs.loc[dfs.code_macro == macro].geocode.unique()
    state = dfs.loc[dfs.code_macro == macro].state.values[0]

    return geocodes, state

In [5]:
def split_geocodes(geocodes):
    
    '''
    This function split the geocodes between the cities with populations up and below 30k in 2022.
    
    :param geocode:list of int. A list with seven-digit ibge codes for brazilian cities 
     
    '''
    
    dfpop = pd.read_csv('poptcu2010-2022_rgi.csv')

    g_low = dfpop.loc[ (dfpop.CODMUN7.isin(geocodes)) & (dfpop.POP22 <= 30000) ].CODMUN7.unique()
    
    g_up = np.setdiff1d(geocodes, g_low)
    
    if geocodes.shape[0] != g_low.shape[0] + g_up.shape[0]:
    
        print('Error subtracting geocodes')
    
    return g_up, g_low

In [6]:
def transform_data(df, geocode, geo_col = 'municipio_geocodigo'): 
    '''
    This filters the data for a specific region and returns it as a separate dataframe.
    
    :param df: pd.DataFrame.
    :param geocode:. Must be at the same type of the geo_col 
    :param geo_col: str. Name of the column in the df that it will be used to filter the geocode value
     
    '''
        
    
    df_ep = df.loc[df[geo_col] == geocode]
    
    del df_ep[geo_col]
    
    df_ep.columns = df_ep.columns + f'_{geocode}'
    
    return df_ep

In [27]:
predictors_clim = ['temp_min', 'temp_max', 'umid_min', 'umid_max',
                   'pressao_min', 'pressao_max', 'precip_tot', 'rainy_days',
                   'temp_mean', 'temp_amp','umid_mean','umid_amp',
                   'pressao_mean']
def predictors_ep_macro(macro): 
    '''
    This function is used to organize in a table the epidemiological predictors related to a specific health macroregion
    
    :params macro: int. A four digit number
    '''
    
    geocodes, state = get_geocodes_and_state(macro)

    # get epidemiological factors 
    df_ep = pd.read_parquet(f'{PATH}/cases/{state}_dengue.parquet',
                           columns = ['data_iniSE', 'casos_est', 'municipio_geocodigo', 'p_rt1', 'Rt', 'p_inc100k'])
    
    # select only the geocodes include in the health macroregion
    df_ep = df_ep.loc[df_ep.municipio_geocodigo.isin(geocodes)]
    
    df_ep = df_ep.sort_index()
    
    # split the geocodes between cities with population up and below 30k in 2022
    g_up, g_low = split_geocodes(geocodes)    

    # get the data of each city with population above 30k
    list_data_ep = []

    for g in g_up:

        list_data_ep.append(transform_data(df_ep, g))
    
    # get the total weekly cases of this health macroregion 
    data_macro_ep = df_ep[['casos_est']].resample('W-SUN').sum()#.agg({'casos_est':np.sum, 
                                                      #'p_rt1': np.mean, 
                                                      #'Rt': np.mean})

    data_macro_ep.columns = data_macro_ep.columns + f'_{macro}'

    list_data_ep.append(data_macro_ep)
    
    
    # aggregate the data from small cities
    
    data_small_cities = df_ep.loc[df_ep.municipio_geocodigo.isin(g_low)][['casos_est','p_rt1', 'Rt']].resample('W-SUN').agg({'casos_est':np.sum, 
                                                                                                        'p_rt1': np.mean, 'Rt': np.mean})
    data_small_cities.columns = data_small_cities.columns + f'_small'
    
    list_data_ep.append(data_small_cities)
    
    data_ep = pd.concat(list_data_ep, axis=1, join='outer')#.fillna(method='ffill')
    
    return data_ep 


def predictors_clim_macro(macro):
    '''
    This function is used to organize in a table the climate predictors related to a specific health macroregion
    
    :params macro: int. A four digit number
    '''
    geocodes, state = get_geocodes_and_state(macro)
    
    # get climate factors 
    df_clim = pd.read_parquet(f'../data/climate/{state}_climate.parquet',
                         columns = predictors_clim.append('geocodigo'))
    
    # select only the geocodes include in the health macroregion
    df_clim = df_clim.loc[df_clim.geocodigo.isin(geocodes)]

    df_clim = df_clim.loc[df_clim.index.year >= 2010]

    del df_clim['index']
    
    # compute other climate features 
    df_clim['temp_mean'] = (df_clim.temp_max+df_clim.temp_min)/2

    df_clim['pressao_mean'] = (df_clim.pressao_max+df_clim.pressao_min)/2

    df_clim['umid_mean'] = (df_clim.umid_max+df_clim.umid_min)/2

    df_clim['temp_amp'] = df_clim.temp_max-df_clim.temp_min
        # Rainy days
    df_clim['rainy_days'] = df_clim.precip_max > 0
        # Humidity amplitude
    df_clim['umid_amp'] = df_clim.umid_max - df_clim.umid_min

    # agg data by weekly since that's the time scale of the cases 
    df_clim = df_clim.groupby('geocodigo').resample('W-SUN').agg({'temp_min':np.mean, 'temp_max': np.mean,

                                                                'umid_min':np.mean, 'umid_max': np.mean,
                                                                'pressao_min':np.mean, 'pressao_max': np.mean,
                                                                'precip_tot':np.sum, 'rainy_days': np.sum,
                                                                'temp_mean':np.mean, 'temp_amp':np.mean,
                                                                'umid_mean': np.mean,'umid_amp': np.mean,
                                                                'pressao_mean':np.mean}).reset_index().set_index('date')
    
    # split the geocodes between cities with population up and below 30k in 2022
    g_up, g_low = split_geocodes(geocodes)
    
    
    # get the predictors of each city with population above 30k
    
    list_data_clim = []

    for g in g_up:

        list_data_clim.append(transform_data(df_clim, g, 'geocodigo'))

    #del df_clim['geocodigo']

    #data_macro_clim = df_clim.resample('W-SUN').agg({'temp_min':np.mean, 'temp_max': np.mean,

     #                                                           'umid_min':np.mean, 'umid_max': np.mean,
      #                                                          'pressao_min':np.mean, 'pressao_max': np.mean,
       #                                                         'precip_tot':np.sum, 'rainy_days': np.sum,
        #                                                        'temp_mean':np.mean, 'temp_amp':np.mean,
         #                                                       'umid_mean': np.mean,'umid_amp': np.mean,
          #                                                      'pressao_mean':np.mean}).reset_index().set_index('date')

    #data_macro_clim.columns = data_macro_clim.columns + f'_{macro}'

    #list_data_clim.append(data_macro_clim)
    
    # aggregate the data from small cities and save the mean as predictor
    
    data_small_cities = df_clim.loc[df_clim.geocodigo.isin(g_low)][['temp_min','temp_max',
                                                                'umid_min', 'umid_max',
                                                                'pressao_min', 'pressao_max',
                                                                'precip_tot', 'rainy_days',
                                                                'temp_mean', 'temp_amp',
                                                                'umid_mean', 'umid_amp',
                                                                'pressao_mean']].resample('W-SUN').mean()
    

    data_small_cities.columns = data_small_cities.columns + f'_small'
    
    list_data_clim.append(data_small_cities)
    
    data_clim = pd.concat(list_data_clim, axis=1, join='outer').fillna(method='ffill')
    
    return data_clim 


def get_data_macro(macro):
    '''
    This function is used to organize in a table the climate and epidemiological predictors 
    related to a specific health macroregion.
    
    :params macro: int. A four-digit number
    '''
    
    data_ep = predictors_ep_macro(macro)
    data_ep = data_ep.loc[~data_ep.index.duplicated(keep='first')]
    
    data_clim = predictors_clim_macro(macro)
    # print(data_ep.index.is_unique, data_clim.index.is_unique)
    return pd.concat([data_ep, data_clim], axis = 1, join = 'outer')


def predictors_ep_state(state): 
    
    '''
    This function is used to organize in a table the epidemiological predictors related to a specific state    
    :params state: str. Two leters code 
    '''
        
    
    # get epidemiological factors 
    df_ep = pd.read_parquet(f'../data/cases/{state}_dengue.parquet',
                           columns = ['data_iniSE', 'casos_est', 'municipio_geocodigo', 'p_rt1', 'Rt'])

    df_ep = df_ep.sort_index()
    
    # this copy will be used to compute the target for all the state later 
    df_ep_copy = df_ep.copy()
    
    # link the geocode and the health macroregion code 
    df_ep = df_ep.reset_index().merge(dfs[['code_macro', 'geocode']].rename(columns = {'geocode':'municipio_geocodigo'}),
                          on = 'municipio_geocodigo').set_index('data_iniSE')
    
    del df_ep['municipio_geocodigo']
    
    # resample the data based of the macroregion 
    df_ep = df_ep.groupby('code_macro').resample('W-SUN').agg({'casos_est':np.sum, 
                               'p_rt1': np.mean, 
                               'Rt': np.mean}).reset_index().set_index('data_iniSE')
    
    df_ep.index = pd.to_datetime(df_ep.index)
    
    # transform in column the data of each predictor by macroregion
    list_data_ep = []

    for m in df_ep.code_macro.unique():

        list_data_ep.append(transform_data(df_ep, m, 'code_macro'))
    
    # get the total weekly cases of the state (it will be used as target)
    data_state_ep = df_ep_copy[['casos_est']].resample('W-SUN').sum()#agg({'casos':np.sum, 
                               #'p_rt1': np.mean, 
                               #'Rt': np.mean})

    data_state_ep.columns = data_state_ep.columns + f'_{state}'

    list_data_ep.append(data_state_ep)
    
    # final dataframe
    data_ep = pd.concat(list_data_ep, axis=1, join='outer')#.fillna(method='ffill')
    
    return data_ep 

def predictors_clim_state(state):
    '''
    This function is used to organize in a table the climate predictors related to a specific state    
    :params state: str. Two leters code 
    '''
    
    # get climate factors 
    df_clim = pd.read_parquet(f'../data/climate/{state}_climate.parquet',
                         columns = predictors_clim.append('geocodigo'))

    df_clim = df_clim.loc[df_clim.index.year >= 2010]
    
    del df_clim['index']

    df_clim['temp_mean'] = (df_clim.temp_max+df_clim.temp_min)/2

    df_clim['pressao_mean'] = (df_clim.pressao_max+df_clim.pressao_min)/2

    df_clim['umid_mean'] = (df_clim.umid_max+df_clim.umid_min)/2

    df_clim['temp_amp'] = df_clim.temp_max-df_clim.temp_min
        # Rainy days
    df_clim['rainy_days'] = df_clim.precip_max > 0
        # Humidity amplitude
    df_clim['umid_amp'] = df_clim.umid_max - df_clim.umid_min

    # link the geocode and the health macroregion code 
    
    df_clim = df_clim.reset_index().merge(dfs[['code_macro', 'geocode']].rename(columns = {'geocode':'geocodigo'}),
                          on = 'geocodigo').set_index('date')

    del df_clim['geocodigo']

    # resample the data based of the macroregion 
    df_clim = df_clim.groupby('code_macro').resample('W-SUN').agg({'temp_min':np.mean, 'temp_max': np.mean,

                                                                'umid_min':np.mean, 'umid_max': np.mean,
                                                                'pressao_min':np.mean, 'pressao_max': np.mean,
                                                                'precip_tot':np.sum, 'rainy_days': np.sum,
                                                                'temp_mean':np.mean, 'temp_amp':np.mean,
                                                                'umid_mean': np.mean,'umid_amp': np.mean,
                                                                'pressao_mean':np.mean}).reset_index().set_index('date')

    # transform in column the data of each predictor by macroregion
    list_data_clim = []

    for m in df_clim.code_macro.unique():

        list_data_clim.append(transform_data(df_clim, m, 'code_macro'))
    

    #data_state_clim = df_clim_copy.resample('W-SUN').agg({'temp_min':np.mean, 'temp_max': np.mean,

     #                                                           'umid_min':np.mean, 'umid_max': np.mean,
      #                                                          'pressao_min':np.mean, 'pressao_max': np.mean,
       #                                                         'precip_tot':np.sum, 'rainy_days': np.sum,
        #                                                        'temp_mean':np.mean, 'temp_amp':np.mean,
         #                                                       'umid_mean': np.mean,'umid_amp': np.mean,
          #                                                      'pressao_mean':np.mean}).reset_index().set_index('date')

    #data_state_clim.columns = data_state_clim.columns + f'_{state}'

    #list_data_clim.append(data_state_clim)

    # final dataframe
    data_clim = pd.concat(list_data_clim, axis=1, join='outer').ffill()#.fillna(method='ffill')
    
    return data_clim 


def get_data_state(state):
    '''
    This function is used to organize in a table the climate and epidemiological predictors 
    related to a specific state.
    
    :params macro: int. A four digit number
    '''
    
    data_ep = predictors_ep_state(state)
    
    data_clim = predictors_clim_state(state)
    
    return pd.concat([data_ep, data_clim], axis = 1, join = 'outer')#.fillna(method = 'ffill')



## Get data for all macro in MG: 

In [28]:

# for macro in dfs.loc[dfs.state=='MG'].code_macro.unique():
for macro in dfs.code_macro.unique():

    df1 = get_data_macro(macro)
    
    df1.to_csv(f'../data/dengue_{macro}.csv')
    
    df1.head()

/tmp/ipykernel_757098/3285190035.py:45: FutureWarning: The provided callable <function sum at 0x7fd5f4640180> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_small_cities = df_ep.loc[df_ep.municipio_geocodigo.isin(g_low)][['casos_est','p_rt1', 'Rt']].resample('W-SUN').agg({'casos_est':np.sum,
/tmp/ipykernel_757098/3285190035.py:45: FutureWarning: The provided callable <function mean at 0x7fd5f4641260> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  data_small_cities = df_ep.loc[df_ep.municipio_geocodigo.isin(g_low)][['casos_est','p_rt1', 'Rt']].resample('W-SUN').agg({'casos_est':np.sum,
/tmp/ipykernel_757098/3285190035.py:89: FutureWarning: The provided callable <function mean at 0x7fd5f4641260> is currently using DatetimeIndexResample

In [9]:
state = 'MG'

df2 = get_data_state(state)

df2.head()

/tmp/ipykernel_757098/3741580052.py:184: FutureWarning: The provided callable <function sum at 0x7fd5f4640180> is currently using DatetimeIndexResamplerGroupby.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_ep = df_ep.groupby('code_macro').resample('W-SUN').agg({'casos_est':np.sum,
/tmp/ipykernel_757098/3741580052.py:184: FutureWarning: The provided callable <function mean at 0x7fd5f4641260> is currently using DatetimeIndexResamplerGroupby.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_ep = df_ep.groupby('code_macro').resample('W-SUN').agg({'casos_est':np.sum,
/tmp/ipykernel_757098/3741580052.py:184: FutureWarning: The provided callable <function mean at 0x7fd5f4641260> is currently using DatetimeIndexResamplerGroupby.mean. In a future version of pandas, the provided callable will be used directly

,casos_est_3101,p_rt1_3101,Rt_3101,casos_est_3102,p_rt1_3102,Rt_3102,casos_est_3103,p_rt1_3103,Rt_3103,casos_est_3104,...,umid_max_3114,pressao_min_3114,pressao_max_3114,precip_tot_3114,rainy_days_3114,temp_mean_3114,temp_amp_3114,umid_mean_3114,umid_amp_3114,pressao_mean_3114
2010-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.103580,0.999013,1.002990,23.623390,105,25.975808,10.060156,71.815624,40.575911,1.001001
2010-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,91.234251,0.997709,1.001661,144.832569,245,26.172300,9.370447,73.524498,35.419506,0.999685
2010-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,88.154297,0.999295,1.003492,41.721230,245,26.955346,10.111888,68.444813,39.418967,1.001393
2010-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,87.084402,0.997927,1.002659,58.044680,245,27.029704,10.326734,67.026808,40.115189,1.000293
2010-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,84.693930,0.998273,1.002670,40.796540,245,26.911134,10.386157,65.211321,38.965219,1.000471


In [42]:
df2.to_csv(f'../data/dengue_{state}.csv')

In [43]:
df2['casos_est_MG']

2010-01-03    1461
2010-01-10    2131
2010-01-17    2682
2010-01-24    3899
2010-01-31    4305
              ... 
2023-06-25    4270
2023-07-02    3099
2023-07-09    2298
2023-07-16    1382
2023-07-23     292
Freq: W-SUN, Name: casos_MG, Length: 708, dtype: int64